This notebook focuse on the Training the T5-large model and saving it to the Google bucket as suggested novasearch and Training T5 model in the orginal github repository. As done in the novasearch github we train and test on the canard dataset

In [ ]:
#Installing the dependencies
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5
!pip install -q t5 tensorflow-text==2.2.
!pip install -U tensorflow-gcs-config==2.1.3
!pip install tensorflow_io-gcs-config==0.23.1
!pip install tensorflow==2.3 t5 tensorflow-text==2.3
#%env USE_AUTH_EPHEM=0
import functools
import os
import sys
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5
import t5.models
import seqio

# Required to fix Colab flag parsing issue.
sys.argv = sys.argv[:1]

BASE_DIR = "gs://trecuofg" #@param { type: "string" }
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
DATA_DIR = os.path.join(BASE_DIR, "data/canard_data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
ON_CLOUD = True


if ON_CLOUD:
  print("Setting up GCS access...")
  # Use legacy GCS authentication method.
  os.environ['USE_AUTH_EPHEM'] = '0'
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "v2-8"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.enable_eager_execution()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Installing dependencies...
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     |████████████████████████████████| 153 kB 5.4 MB/s 
     |████████████████████████████████| 116 kB 50.2 MB/s 
     |████████████████████████████████| 385 kB 44.0 MB/s 
     |████████████████████████████████| 306 kB 53.5 MB/s 
     |████████████████████████████████| 1.3 MB 32.1 MB/s 
     |████████████████████████████████| 4.7 MB 41.9 MB/s 
     |████████████████████████████████| 4.6 MB 48.0 MB/s 
     |████████████████████████████████| 4.5 MB 32.1 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
     |████████████████████████████████| 101 kB 8.1 MB/s 
     |████████████████████████████████| 6.6 MB 44.8 MB/s 
     |████████████████████████████████| 511.7 MB 5.0 kB/s 
     |████████████████████████████████| 5.8 MB 17.0 MB/s 
     |████████████████████████████████| 438 kB 16.7 MB/s 
     |████████████████████████████████| 1.6 MB 22.7 MB/s 
     |███████████████████████████

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# Getting the paths from Google bucket
import gzip
import json


canard_tsv_path = {
    "train": os.path.join(DATA_DIR, "training_t5_canard_data.tsv"),
    "validation": os.path.join(DATA_DIR, "validation_t5_canard_data.tsv"),
    "test": os.path.join(DATA_DIR, "test_t5_canard_data.tsv"),
    "cast_test": os.path.join(DATA_DIR, "trec_cast_evaluation_predicted_t5.tsv")
}

In [ ]:
#Splitting the dataset 
def canard_dataset_fn(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(canard_tsv_path[split])
  # Split each "<context>\t<rewrite>" example into (context, rewrite) tuple.
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  # Map each tuple to a {"context": ... "rewrite": ...} dict.
  ds = ds.map(lambda *ex: dict(zip(["context", "rewrite"], ex)))
  return ds

print("A few raw validation examples...")
for ex in tfds.as_numpy(canard_dataset_fn("validation").take(5)):
  print(ex)



A few raw validation examples...
{'context': b'What group disbanded? [CTX] Frank Zappa [TURN] Disbandment', 'rewrite': b'What group disbanded?'}
{'context': b'When did they disband? [CTX] Frank Zappa [TURN] Disbandment [TURN] What group disbanded? [TURN] Zappa and the Mothers of Invention', 'rewrite': b'When did Zappa and the Mothers of Invention disband?'}
{'context': b'What kind of music did they play? [CTX] Frank Zappa [TURN] Disbandment [TURN] What group disbanded? [TURN] Zappa and the Mothers of Invention [TURN] When did they disband? [TURN] In late 1969, Zappa broke up the band.', 'rewrite': b'What kind of music did Zappa and the Mothers of Invention play?'}
{'context': b'Why did they break up? [CTX] Frank Zappa [TURN] Disbandment [TURN] What group disbanded? [TURN] Zappa and the Mothers of Invention [TURN] When did they disband? [TURN] In late 1969, Zappa broke up the band. [TURN] What kind of music did they play? [TURN] major influence on the development of the jazz-rock fusion

In [ ]:
# Preprocessing Canard Dataset
def canard_preprocessor(ds):
  def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    #text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    return text

  def to_inputs_and_targets(ex):
    """Map {"context": ..., "rewrite": ...}->{"inputs":w ..., "targets": ...}."""
    return {
        "inputs":
             tf.strings.join(
                 ["canard context: ", normalize_text(ex["context"])]),
        "targets": normalize_text(ex["rewrite"])
    }
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
# creating a task registery all mix
t5.data.TaskRegistry.add(
    "all_mix",
    dataset_fn=canard_dataset_fn,
    splits=["train", "validation", "test", "cast_test"],
    text_preprocessor=[canard_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.accuracy, t5.evaluation.metrics.bleu, t5.evaluation.metrics.rouge], 
)

In [ ]:
#Setting up the model
MODEL_SIZE = "large" #@param["small", "base", "large", "3B", "11B"]
FOLDER_TO_STORE = "_temperature_0" #@param { type: "string" }

# Public GCS path for T5 pre-trained model checkpoints
BASE_PRETRAINED_DIR = "gs://t5-data/pretrained_models"
PRETRAINED_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_SIZE + FOLDER_TO_STORE)

if ON_CLOUD and MODEL_SIZE == "3B":
  tf.logging.warning(
      "The `3B` model is too large to use with the 5GB GCS free tier. "
      "Make sure you have at least 25GB on GCS before continuing."
  )
elif ON_CLOUD and MODEL_SIZE == "11B":
  raise ValueError(
      "The `11B` parameter is too large to fine-tune on the `v2-8` TPU "
      "provided by Colab. Please comment out this Error if you're running "
      "on a larger TPU."
  )

# Set parallelism and batch size to fit on v2-8 TPU 

model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)
# The models from this paper are based on the Mesh Tensorflow Transformer.
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 256, "targets": 64},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
#Finetuning the steps by setting it at 5000
FINETUNE_STEPS = 5000 #@param {type: "integer"}

model.finetune(
    mixture_or_task_name="all_mix",
    pretrained_model_dir=PRETRAINED_DIR,
    finetune_steps=FINETUNE_STEPS
)

INFO:root:system_path_file_exists:gs://t5-data/pretrained_models/large/operative_config.gin
ERROR:root:Path not found: gs://t5-data/pretrained_models/large/operative_config.gin
INFO:root:Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).


In [ ]:
#Instal necessary dependancy
!pip install numpy==1.19.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.19.3-cp37-cp37m-manylinux2010_x86_64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.3 which is incompatible.
tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.3 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.4.1 which is incompatible.
cmdstanpy 1.0.4 requires numpy>=1.21, but you have numpy 1.19.3 which is incompatible.


In [ ]:
# Using a larger batch size for evaluation, which requires less memory.
model.batch_size = train_batch_size 
model.eval(
    mixture_or_task_name="all_mix",
    checkpoint_steps="all",
    split="validation"
)

INFO:root:system_path_file_exists:gs://trecuofg/models/large_temperature_0/operative_config.gin
ERROR:root:Path not found: gs://trecuofg/models/large_temperature_0/operative_config.gin
INFO:absl:Adding task 'all_mix' with predict metric_fn(s).
/usr/local/lib/python3.7/dist-packages/seqio/utils.py:780: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  num_parallel_calls=num_parallel_calls)
INFO:absl:Skipping packing/padding for 'all_mix' since sequence length is None.
INFO:absl:Setting sequence lengths to {'inputs': 607, 'targets': 75}
INFO:absl:Evaluating checkpoint step: 1000700
INFO:absl:Padding 'all_mix' with sequence lengths: {'inputs': 607, 'targets': 75}
SimdMeshImpl ignoring devices ['', '', '', '', '', '', '', '']
Using default tf glorot_uniform_initializer for variable encoder/block_000/layer_000/SelfAttention/relative_attention_bias  The initial

In [ ]:
# Using a larger batch size for evaluation, which requires less memory.
model.batch_size = train_batch_size 
model.eval(
    mixture_or_task_name="all_mix",
    checkpoint_steps="all",
    split="test"
)

INFO:root:system_path_file_exists:gs://trecuofg/models/large_temperature_0/operative_config.gin
ERROR:root:Path not found: gs://trecuofg/models/large_temperature_0/operative_config.gin
INFO:absl:Adding task 'all_mix' with predict metric_fn(s).
/usr/local/lib/python3.7/dist-packages/seqio/utils.py:780: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  num_parallel_calls=num_parallel_calls)
INFO:absl:Skipping packing/padding for 'all_mix' since sequence length is None.
INFO:absl:Setting sequence lengths to {'inputs': 589, 'targets': 120}
INFO:absl:Evaluating checkpoint step: 1000700
INFO:absl:Padding 'all_mix' with sequence lengths: {'inputs': 589, 'targets': 120}
SimdMeshImpl ignoring devices ['', '', '', '', '', '', '', '']
Using default tf glorot_uniform_initializer for variable encoder/block_000/layer_000/SelfAttention/relative_attention_bias  The initi

In [ ]:
#Testing it on the example questions provided
question_1 = "Did they have any clues? [CTX] Anna Politkovskaya [TURN] The murder remains unsolved, 2016" #@param {type:"string"}
question_2 = "How did they target her email? [CTX] Anna Politkovskaya [TURN] The murder remains unsolved, 2016 [TURN] Did they have any clues? [TURN] probably FSB) are known to have targeted the webmail account of the murdered Russian journalist Anna Politkovskaya." #@param {type:"string"}
question_3 = "Did they have any murder suspects? [CTX] Anna Politkovskaya [TURN] The murder remains unsolved, 2016 [TURN] Did they have any clues? [TURN] probably FSB) are known to have targeted the webmail account of the murdered Russian journalist Anna Politkovskaya. [TURN] How did they target her email? [TURN] On 5 December 2005, RFIS initiated an attack against the account annapolitovskaya@US Provider1, by deploying malicious software [TURN] Did they get into trouble for that? [TURN] I don't know." #@param {type:"string"}
question_4 = "Is there anything else interesting in the article? [CTX] Anna Politkovskaya [TURN] The murder remains unsolved, 2016 [TURN] Did they have any clues? [TURN] probably FSB) are known to have targeted the webmail account of the murdered Russian journalist Anna Politkovskaya. [TURN] How did they target her email? [TURN] On 5 December 2005, RFIS initiated an attack against the account annapolitovskaya@US Provider1, by deploying malicious software [TURN] Did they get into trouble for that? [TURN] I don't know. [TURN] Did they have any murder suspects? [TURN] After the three Makhmudov brothers, Khadjikurbanov and Lom-Ali Gaitukayev were convicted in 2014, [TURN] Did they go to jail? [TURN] I don't know." #@param {type:"string"}

questions = [question_1, question_2, question_3, question_4]

now = time.time()
# Write out the  questions to text files.
predict_inputs_path = os.path.join(MODEL_DIR, "predict_inputs_%d.txt" % now)
predict_outputs_path = os.path.join(MODEL_DIR, "predict_outputs_%d.txt" % now)
# Manually apply preprocessing by prepending "canard context:".
with tf.io.gfile.GFile(predict_inputs_path, "w") as f:
  for q in questions:
    f.write("canard context: %s\n" % q.lower())

# Ignore any logging.
with tf_verbosity_level('ERROR'):
  model.batch_size = 4
  model.predict(
      input_file=predict_inputs_path,
      output_file=predict_outputs_path,
      # Select the most probable output token at each step.
      temperature=0,
  )

# The output filename will have the checkpoint appended so we glob to get 
# the latest.
prediction_files = sorted(tf.io.gfile.glob(predict_outputs_path + "*"))
print("\nPredictions using checkpoint %s:\n" % prediction_files[-1].split("-")[-1])
with tf.io.gfile.GFile(prediction_files[-1]) as f:
  for q, a in zip(questions, f):
    if q:
      print("Context: " + q)
      print("rewrite: " + a)
      print()

In [ ]:
#Exporting Model to the google bucket
export_dir = os.path.join(MODEL_DIR, "export")

model.batch_size = 1 # make one prediction per call
saved_model_path = model.export(
    export_dir,
    checkpoint_step=-1,  # use most recent
    beam_size=1,  # no beam search
    temperature=0.0,  # sample according to predicted distribution
)
print("Model saved to:", saved_model_path)

INFO:root:system_path_file_exists:gs://trecuofg/models/large_temperature_0/operative_config.gin
ERROR:root:Path not found: gs://trecuofg/models/large_temperature_0/operative_config.gin
eval_on_tpu ignored because use_tpu is False.
/usr/local/lib/python3.7/dist-packages/mesh_tensorflow/transformer/dataset.py:380: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  return dataset.map(my_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_less_equal/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py", line 201, in wrapper
    return target(*args, **kwargs)  File "/usr/local/lib/python3.7/dist-packages/tens

Model saved to: b'gs://trecuofg/models/large_temperature_0/export/1660202739'


In [ ]:
%tensorflow_version 2.x
!pip install tensorflow-text
from google.colab import auth
auth.authenticate_user()

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 34.1 MB/s 
     |████████████████████████████████| 511.7 MB 5.8 kB/s 
     |████████████████████████████████| 1.6 MB 54.5 MB/s 
     |████████████████████████████████| 438 kB 75.1 MB/s 
     |████████████████████████████████| 5.8 MB 33.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attemptin

In [ ]:
#Run the Exported model
import tensorflow as tf
import tensorflow_text  

# TODO change this to load a different model
saved_model_path = "gs://trecuofg/models/large_temperature_0/export/1660202739"

def load_predict_fn(model_path):
  if tf.executing_eagerly():
    print("Loading SavedModel in eager mode.")
    imported = tf.saved_model.load(model_path, ["serve"])
    return lambda x: imported.signatures['serving_default'](tf.constant(x))['outputs'].numpy()
  else:
    print("Loading SavedModel in tf 1.x graph mode.")
    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], model_path)
    signature_def = meta_graph_def.signature_def["serving_default"]
    return lambda x: sess.run(
        fetches=signature_def.outputs["outputs"].name, 
        feed_dict={signature_def.inputs["input"].name: x}
    )

predict_fn = load_predict_fn(saved_model_path)

Loading SavedModel in eager mode.


In [ ]:
# function for predicting the queries
def answer(question):
  return predict_fn([question])[0].decode('utf-8')

In [ ]:
#Executing the example queries
targets =  [
           "Aside from the discontinued case against a convicted killer, how is the murder of Anna Politkovskaya similar to any other cases?",
           "Why did Superstar Billy Graham return to the WWWF?",
           "What was Superstar Billy Graham's agreement with McMahon?",
           "How did people respond to Superstar Billy Graham's return?"
]

questions = ["canard context: Is it similar to any other cases? [CTX] Anna Politkovskaya [TURN] The murder remains unsolved, 2016 [TURN] Did they have any clues? [TURN] probably FSB) are known to have targeted the webmail account of the murdered Russian journalist Anna Politkovskaya. [TURN] How did they target her email? [TURN] On 5 December 2005, RFIS initiated an attack against the account annapolitovskaya@US Provider1, by deploying malicious software [TURN] Did they get into trouble for that? [TURN] I don't know. [TURN] Did they have any murder suspects? [TURN] After the three Makhmudov brothers, Khadjikurbanov and Lom-Ali Gaitukayev were convicted in 2014, [TURN] Did they go to jail? [TURN] I don't know. [TURN] Is there anything else interesting in the article? [TURN] In accordance with Russian law there is a 15-year statute of limitation for the 'particularly grave' crime of first degree murder. [TURN] Are they close to solving it? [TURN] In May that year the case against him was discontinued because the statute of limitations had expired.",
             "canard context: Why did he return to the WWWF? [CTX] Superstar Billy Graham [TURN] Return to WWWF (1977-1981)",
             "canard context: What was his agreement with McMahon? [CTX] Superstar Billy Graham [TURN] Return to WWWF (1977-1981) [TURN] Why did he return to the WWWF? [TURN] an agreement with promoter Vincent J. McMahon (Senior",
             "canard context: How did people respond to his return? [CTX] Superstar Billy Graham [TURN] Return to WWWF (1977-1981) [TURN] Why did he return to the WWWF? [TURN] an agreement with promoter Vincent J. McMahon (Senior [TURN] What was his agreement with McMahon? [TURN] I don't know."
            ]

def test_examples():
  for i, question in enumerate(questions):
      print("Target: " + targets[i])
      print("Rewrite: " + answer(question))
      print()

In [ ]:
for j in range(1):
  test_examples()

Target: Aside from the discontinued case against a convicted killer, how is the murder of Anna Politkovskaya similar to any other cases?
Rewrite: Is the murder of Anna Politkovskaya similar to any other cases?

Target: Why did Superstar Billy Graham return to the WWWF?
Rewrite: Why did Billy Graham return to the WWWF?

Target: What was Superstar Billy Graham's agreement with McMahon?
Rewrite: What was Billy Graham's agreement with McMahon?

Target: How did people respond to Superstar Billy Graham's return?
Rewrite: How did people respond to Billy Graham's return to the WWWF?

